In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
import json
import pickle
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import pandas as pd
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [26]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

intents = pd.read_json("/content/job_intents.json")

In [27]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        w = nltk.word_tokenize(pattern)
        words.extend(w)

        documents.append((w, intent['tag']))


        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [28]:
lemmatizer = WordNetLemmatizer()

In [29]:
lm_words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

In [30]:
lm_words = sorted(list(set(lm_words)))
lm_words

["'s",
 ',',
 'about',
 'anyone',
 'are',
 'awesome',
 'bye',
 'can',
 'day',
 'do',
 'ekse',
 'fact',
 'for',
 'give',
 'good',
 'goodbye',
 'hello',
 'help',
 'helpful',
 'helping',
 'hemanth',
 'hey',
 'hi',
 'hola',
 'how',
 'interesting',
 'is',
 'know',
 'later',
 'me',
 'more',
 'name',
 'ok',
 'purpose',
 'see',
 'some',
 'something',
 'tell',
 'thank',
 'thanks',
 'that',
 'there',
 'ungubani',
 'what',
 'whats',
 'who',
 'you',
 'your',
 'yourself']

In [31]:
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(lm_words), "unique lemmatized words", lm_words)

53 documents
7 classes ['goodbye', 'greeting', 'hemanth_facts', 'hemanth_info', 'name', 'options', 'thanks']
49 unique lemmatized words ["'s", ',', 'about', 'anyone', 'are', 'awesome', 'bye', 'can', 'day', 'do', 'ekse', 'fact', 'for', 'give', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hemanth', 'hey', 'hi', 'hola', 'how', 'interesting', 'is', 'know', 'later', 'me', 'more', 'name', 'ok', 'purpose', 'see', 'some', 'something', 'tell', 'thank', 'thanks', 'that', 'there', 'ungubani', 'what', 'whats', 'who', 'you', 'your', 'yourself']


In [32]:
pickle.dump(lm_words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [33]:
training = []
output_empty = [0] * len(classes)

In [34]:
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in lm_words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [35]:
random.shuffle(training)
training = np.array(training, dtype='object')

In [36]:
training = np.array(training)
training.shape

(53, 2)

In [37]:
train_x = list(training[:,0])
train_y = list(training[:,1])

VALIDATION_SET = (train_x, train_y)

In [38]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [39]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [40]:
history = model.fit(np.array(train_x), np.array(train_y), epochs=250, validation_data = VALIDATION_SET, batch_size=5, verbose=1)

Epoch 1/250
11/11 [==============================] - 1s 27ms/step - loss: 1.9768 - accuracy: 0.1321 - val_loss: 1.8768 - val_accuracy: 0.2453
Epoch 2/250
11/11 [==============================] - 0s 11ms/step - loss: 1.8681 - accuracy: 0.2830 - val_loss: 1.7562 - val_accuracy: 0.2642
Epoch 3/250
11/11 [==============================] - 0s 8ms/step - loss: 1.7866 - accuracy: 0.2075 - val_loss: 1.6370 - val_accuracy: 0.3396
Epoch 4/250
11/11 [==============================] - 0s 8ms/step - loss: 1.7529 - accuracy: 0.2830 - val_loss: 1.5208 - val_accuracy: 0.4340
Epoch 5/250
11/11 [==============================] - 0s 12ms/step - loss: 1.5634 - accuracy: 0.3585 - val_loss: 1.3983 - val_accuracy: 0.6226
Epoch 6/250
11/11 [==============================] - 0s 8ms/step - loss: 1.4818 - accuracy: 0.4717 - val_loss: 1.2639 - val_accuracy: 0.7170
Epoch 7/250
11/11 [==============================] - 0s 13ms/step - loss: 1.4491 - accuracy: 0.4528 - val_loss: 1.1417 - val_accuracy: 0.7170
Epoch 8/2

In [41]:
model.save('chatbot_model.h5', history)
print("model Saved")

model Saved


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
